# 事前アンケートの質問項目

0. タイムスタンプ
1. 【Q1】What is your age ?
2. 【Q2】What gender do you identify as ?
3. 【Q3】Which geographical region are you currently working or attending school/university in ?
4. 【Q4】Which geographical region do you most strongly associate with ?
5. 【Q5】What is your job title ?
6. 【Q6】Which group do you belong to ?
7. 【Q7】What is your research type ?
8. 【Q8】How long have you been in this field ?
9. 【Q9】Are you satisfied with your career to date ?
10. 【Q10】How many hours, on average, do you spend on housework, childcare, and caregiving per day ?
11. 【Q11】Did you already sign up for the diversity session in ICRC2023?
12. 【Q12】What do you think about the initiatives on DE&I of your group? [Gender balance]
13. 【Q12】What do you think about the initiatives on DE&I of your group? [Diversity]
14. 【Q12】What do you think about the initiatives on DE&I of your group? [Equity]
15. 【Q12】What do you think about the initiatives on DE&I of your group? [Inclusion]
16. 【Q13】What is the percentage of female researcher in your group?
17. 【Q14】What do you think about the percentage above ?
18. 【Q15】Please let us know If your group has any good practice examples related to DE&I ?
19. 【Q16】Please let us know if there is anything your group needs to work on or if your group has any problems related to DE&I.
20. 【Q17】What are your thoughts on diversity, equity & inclusion initiatives ? [Gender balance]
21. 【Q17】What are your thoughts on diversity, equity & inclusion initiatives ? [Diversity]
22. 【Q17】What are your thoughts on diversity, equity & inclusion initiatives ? [Equity]
23. 【Q17】What are your thoughts on diversity, equity & inclusion initiatives ? [Inclusion]
24. 【Q18】Could you tell us more about your thoughts (agree / disagree) ?
25. 【Q19】When did you first become interested in science ?
26. 【Q20】Do you have any concerns / problems related to DE&I initiatives in science ?
27. 【Q21】What reasons do you think are hindering DE&I initiatives in science ?
28. 【Q22】Comments

Pythonのライブラリ
- ``pathlib.Path`` : パス操作
- ``pandas`` : データ集計
- ``altair (v5)`` : プロット作成

In [1]:
from pathlib import Path
import pandas as pd
import altair as alt
import titanite as ti

print(f"Pandas {pd.__version__}")
print(f"Altair {alt.__version__}")
print(f"Titanite {ti.__version__}")

Pandas 2.0.3
Altair 5.0.1
Titanite 0.1.2


事前アンケートの結果を読み込む

- 回答データはGoogleスプレッドシートからCSV形式でダウンロード（手動）
- ファイルパスは``../data/test_data/``にする（あとで変えるかも）
- 回答時刻（``timestamp``カラム）はDateTimeオブジェクトに変換する
- 回答数を集計（主に``sum``）するためのカラム``response``を追加する

In [2]:
f_cfg = "../sandbox/config.toml"
cfg = ti.Config(fname=f_cfg)
cfg.load()
category = cfg.categories()

In [3]:
f_csv = "../data/test_data/all.csv"
data = pd.read_csv(f_csv, parse_dates=["timestamp"])
data = ti.categorical_data(data, category)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279 entries, 0 to 278
Data columns (total 52 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   timestamp          279 non-null    datetime64[ns]
 1   q1                 279 non-null    category      
 2   q2                 279 non-null    category      
 3   q3                 279 non-null    category      
 4   q4                 279 non-null    category      
 5   q5                 279 non-null    category      
 6   q6                 279 non-null    category      
 7   q7                 255 non-null    category      
 8   q8                 279 non-null    category      
 9   q9                 279 non-null    category      
 10  q10                279 non-null    int64         
 11  q11                279 non-null    category      
 12  q12_genderbalance  279 non-null    category      
 13  q12_diversity      279 non-null    category      
 14  q12_equity

ビニングを追加

In [4]:
data["q10_binned"] = pd.cut(
    data["q10"],
    [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 25],
    labels = ["Prefer not to answer", "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10+"],
    right=False)
data["q13_binned"] = pd.cut(
    data["q13"],
    [-1, 0, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100],
    labels = ["Prefer not to answer", "0", "10", "15", "20", "25", "30", "35", "40", "45", "50", "55", "60", "65", "70", "75", "80", "85", "90", "95"],
    right=False)

In [5]:
# データフレームを確認する（不要なときはコメントアウトしておく）
# data.head()
# data.columns

In [6]:
#alt.Chart(data).mark_bar().encode(x="q1", y="count()")

回答時刻を調べてみる

- [バブル図](https://altair-viz.github.io/gallery/histogram_scatterplot.html)を使ってみる → いまいちだったのでヒートマップにした
- ``timestamp``カラムを使って、回答時刻の分布を調べる
- 記録されている日時は、たぶんUTC+0900？（よく分かってない）
- 年月日（=date）ごとにリサンプルして合計したいので、データフレームのインデックスを``timestamp``にする
- ``date``と``時刻``のヒートマップを作りたいので、それ用のカラムを追加する

In [7]:
data.index = data["timestamp"]
tm = data.resample("H")["response"].sum().reset_index()
# tm["yyyymmdd"] = pd.to_datetime(tm["timestamp"].dt.date)
# tm["HH"] = tm["timestamp"].dt.hour
# tm.info()

In [8]:
#tm

回答

In [9]:
base = alt.Chart(tm).encode(
    alt.X("hours(timestamp):T").title("時刻"),
    alt.Y("yearmonthdate(timestamp):T").title("記入日").axis(format="%Y-%m-%d").sort("descending"),
).properties(
    width=800,
    height=400,
)


hm = base.mark_rect().encode(
    alt.Color("response").scale(scheme="blues").title("回答数"),
    #size="response"
)

tt = base.mark_text(dx=15, dy=10, baseline="line-top").encode(
    alt.Text("response"),
)


(hm + tt).save("tmp_registered.png")
(hm + tt)

alt.LayerChart(...)

すべての質問に対してヒストグラムを作成する

- 横軸を「質問番号」にしたヒストグラムを作成する
- 横軸はカテゴリカルなデータにして、任意の順番に並べたい（TODO）
  - いまは自動でアルファベット順に整列

In [10]:
def make_histogram(data: pd.DataFrame, x: str, title: str):
    g = list(set(["q2", x]))
    grouped = data.groupby(g)["response"].sum().reset_index().sort_values(by=x)
    h = alt.Chart(grouped).mark_bar().encode(
        alt.X(x),
        alt.Y("response"),
        alt.Color("q2:N"),
        alt.Order("response", sort="descending"),
    ).properties(
        title=title,
        width=400,
        height=400,
    )

    hs = h.encode(
        alt.Y("response").stack("normalize"),
    )

    chart = h | hs

    return {"data": grouped, "chart": chart}

In [11]:
category["gender"]

CategoricalDtype(categories=['Male', 'Female', 'Non-binary', 'Prefer to self-identify',
                  'Prefer not to answer'],
, ordered=True)

In [12]:
t = cfg.questions.get("q10", "Could not get title")
q = make_histogram(data, "q10_binned", t)
q["data"]
q["chart"]

alt.HConcatChart(...)

In [21]:
# すべてのカラムに対して、2つの総当たりの組み合わせを取得する
# ただし：timestampとresponseは除外する
# ただし：自由記述は除外（極性などは残す）

headers = [
    "q1",
    "q2",
    "q3", "q3_regional", "q3_subregional",
    "q4", "q4_regional", "q4_subregional",
    "q5",
    "q6",
    "q7",
    "q8",
    "q9",
    "q10", "q10_binned",
    "q11",
    "q12_diversity", "q12_equity", "q12_genderbalance", "q12_inclusion",
    "q13", "q13_binned",
    "q14", 
    "q15_polarity", "q15_subjectivity",
    "q16_polarity", "q16_subjectivity",
    "q17_diversity", "q17_equity", "q17_genderbalance", "q17_inclusion",
    "q18_polarity", "q18_subjectivity",
    "q19",
    "q20_polarity", "q20_subjectivity",
    "q21_polarity", "q21_subjectivity",
    "q22_polarity", "q22_subjectivity",
]

In [20]:
for (h1, h2) in matches:
    key = h1.split("_")[0]
    print(h1, h2, key)

q1 q2 q1
q1 q3 q1
q1 q3_regional q1
q1 q3_subregional q1
q1 q4 q1
q1 q4_regional q1
q1 q4_subregional q1
q1 q5 q1
q1 q6 q1
q1 q7 q1
q1 q8 q1
q1 q9 q1
q1 q10 q1
q1 q10_binned q1
q1 q11 q1
q1 q12_diversity q1
q1 q12_equity q1
q1 q12_genderbalance q1
q1 q12_inclusion q1
q1 q13 q1
q1 q13_binned q1
q1 q14 q1
q1 q15_polarity q1
q1 q15_subjectivity q1
q1 q16_polarity q1
q1 q16_subjectivity q1
q1 q17_diversity q1
q1 q17_equity q1
q1 q17_genderbalance q1
q1 q17_inclusion q1
q1 q18_polarity q1
q1 q18_subjectivity q1
q1 q19 q1
q1 q20_polarity q1
q1 q20_subjectivity q1
q1 q21_polarity q1
q1 q21_subjectivity q1
q1 q22_polarity q1
q1 q22_subjectivity q1
q2 q3 q2
q2 q3_regional q2
q2 q3_subregional q2
q2 q4 q2
q2 q4_regional q2
q2 q4_subregional q2
q2 q5 q2
q2 q6 q2
q2 q7 q2
q2 q8 q2
q2 q9 q2
q2 q10 q2
q2 q10_binned q2
q2 q11 q2
q2 q12_diversity q2
q2 q12_equity q2
q2 q12_genderbalance q2
q2 q12_inclusion q2
q2 q13 q2
q2 q13_binned q2
q2 q14 q2
q2 q15_polarity q2
q2 q15_subjectivity q2
q2 q16_polarit

In [23]:
for header in headers:
    key = header.split("_")[0]
    title = cfg.questions.get(key, "Could Not Find Key")
    hist = make_histogram(data, header, title)
    fname = f"../data/quick_summary/tmp_{header}.csv"
    hist["data"].to_csv(fname, index=False)
    fname = f"../data/quick_summary/tmp_{header}.png"
    hist["chart"].save(fname)
    print(f"Saved as {fname}")

Saved as ../data/quick_summary/tmp_q1.png
Saved as ../data/quick_summary/tmp_q2.png
Saved as ../data/quick_summary/tmp_q3.png
Saved as ../data/quick_summary/tmp_q3_regional.png
Saved as ../data/quick_summary/tmp_q3_subregional.png
Saved as ../data/quick_summary/tmp_q4.png
Saved as ../data/quick_summary/tmp_q4_regional.png
Saved as ../data/quick_summary/tmp_q4_subregional.png
Saved as ../data/quick_summary/tmp_q5.png
Saved as ../data/quick_summary/tmp_q6.png
Saved as ../data/quick_summary/tmp_q7.png
Saved as ../data/quick_summary/tmp_q8.png
Saved as ../data/quick_summary/tmp_q9.png
Saved as ../data/quick_summary/tmp_q10.png
Saved as ../data/quick_summary/tmp_q10_binned.png
Saved as ../data/quick_summary/tmp_q11.png
Saved as ../data/quick_summary/tmp_q12_diversity.png
Saved as ../data/quick_summary/tmp_q12_equity.png
Saved as ../data/quick_summary/tmp_q12_genderbalance.png
Saved as ../data/quick_summary/tmp_q12_inclusion.png
Saved as ../data/quick_summary/tmp_q13.png
Saved as ../data/qui

2つの質問をクロス集計する

- すべてのカラムに対してクロス集計する
- CSV集計するときの見出し列をわかりやすくするために、転置（``.T``）している
  - もっとコードが読みやすくなる方法があるかもしれない

In [27]:
import itertools
matches = list(itertools.combinations(headers, 2))
len(matches)

780

In [28]:
ctabs = {}
for h1, h2 in matches:
    ctab = pd.crosstab(data[h1], data[h2])
    name = f"{h1}_{h2}"
    fname = f"../data/quick_summary/tmp_crosstab_{name}.csv"
    ctab.T.to_csv(fname)
    print(f"Saved as {fname}")
    ctabs.update({name: ctab})

Saved as ../data/quick_summary/tmp_crosstab_q1_q2.csv
Saved as ../data/quick_summary/tmp_crosstab_q1_q3.csv
Saved as ../data/quick_summary/tmp_crosstab_q1_q3_regional.csv
Saved as ../data/quick_summary/tmp_crosstab_q1_q3_subregional.csv
Saved as ../data/quick_summary/tmp_crosstab_q1_q4.csv
Saved as ../data/quick_summary/tmp_crosstab_q1_q4_regional.csv
Saved as ../data/quick_summary/tmp_crosstab_q1_q4_subregional.csv
Saved as ../data/quick_summary/tmp_crosstab_q1_q5.csv
Saved as ../data/quick_summary/tmp_crosstab_q1_q6.csv
Saved as ../data/quick_summary/tmp_crosstab_q1_q7.csv
Saved as ../data/quick_summary/tmp_crosstab_q1_q8.csv
Saved as ../data/quick_summary/tmp_crosstab_q1_q9.csv
Saved as ../data/quick_summary/tmp_crosstab_q1_q10.csv
Saved as ../data/quick_summary/tmp_crosstab_q1_q10_binned.csv
Saved as ../data/quick_summary/tmp_crosstab_q1_q11.csv
Saved as ../data/quick_summary/tmp_crosstab_q1_q12_diversity.csv
Saved as ../data/quick_summary/tmp_crosstab_q1_q12_equity.csv
Saved as ..